In [411]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#You need manually fix data in tox24_alogps_oestate.csv, row 1509

#this is causing errors in standardization

#'C1=CC=[N+](C(=C1)[S-])[O-].C1=CC=[N+](C(=C1)[S-])[O-].[ZnH2+2]' -> 
#'C1=CC=N(=O)C([S-])=C1.C1=CC=N(=O)C([S-])=C1.[Zn+2]'

In [412]:
import pandas as pd
import numpy as np

from src.config import mem
N_JOBS = 12
RANDOM_SEED = 42

df = pd.read_csv('../data/raw/tox24_alogps_oestate.csv')

df.drop(['N'], axis=1, inplace=True)
df.drop_duplicates(inplace=True)

print(df.shape)
df.head()

(1511, 367)


,SMILES,ALogPS_logP,ALogPS_logS,HALOG,MW,N.1,NA,NH,O,P,...,Se1C2N2dd,Se1C2N==sd,SdddN,SdddN(nitro),Se2N3O1d,SddSi,Se2O1Si2d,Se1B3C3sa,Se1N3pO1a,SeaC2N3paa
0,CC1=CC[C@H]2C[C@@H]1C2(C)C,3.66,-2.94,0,136.3,0,10,16,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,CC1(C)[C@@H]2C[C@H]1C(=C)CC2,3.94,-3.33,0,136.3,0,10,16,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,COC(=O)C(C)OC1=CC=C(OC2=CC=C(Cl)C=C2Cl)C=C1,5.13,-5.40,2,341.2,0,22,14,4,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,CC(OC1=C(C)C=C(Cl)C=C1)C(O)=O,2.87,-2.47,1,214.7,0,14,11,3,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,CCCCCCCCCCCCCC(=O)OCC(O)CO,4.77,-4.09,0,302.5,0,21,34,4,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [413]:
train = pd.read_csv('../data/raw/tox24_challenge_train.csv')
train.drop_duplicates(subset='SMILES', inplace=True)

test = pd.read_csv('../data/raw/tox24_challenge_test.csv')

In [414]:
len(test), test.SMILES.isin(df.SMILES).sum()

(500, 500)

In [415]:
len(train), train.SMILES.isin(df.SMILES).sum()

(1011, 1011)

In [416]:
len(test) + len(train), len(df)

(1511, 1511)

In [417]:
features = df.columns.tolist()
features.remove('SMILES')
features

['ALogPS_logP',
 'ALogPS_logS',
 'HALOG',
 'MW',
 'N.1',
 'NA',
 'NH',
 'O',
 'P',
 'PSA',
 'S',
 'SdCH2',
 'SdssC',
 'Se1C1C4s',
 'Se1C2C2ss',
 'Se1C2C3sd',
 'Se1C2C3ss',
 'Se1C3C3ds',
 'Se1C3C4ss',
 'Se2C1C3s',
 'SsCH3',
 'SssCH2',
 'SsssCH',
 'SssssC',
 'aCNOS',
 'ACCEPTORS',
 'RBONDS',
 'SaaCH',
 'SaasC',
 'SdO',
 'SdO(ester)',
 'Se1C1C3s',
 'Se1C1O2s',
 'Se1C3Cl1a',
 'Se1C3O2as',
 'Se1C3O2ds',
 'Se1C3O2ss',
 'Se2C3O1s',
 'SeaC2C2aa',
 'SeaC2C3aa',
 'SeaC3C3aa',
 'SsCl',
 'SssO',
 'SssO(ester)',
 'DONORS',
 'Se1C1C2s',
 'Se1C2O1s',
 'Se1C2O2ss',
 'Se1C3O1s',
 'SsOH',
 'SsOH(alc)',
 'Se1C1C3d',
 'Se1C2C4ss',
 'Se1C4C4ss',
 'SdsCH',
 'Se1C2C2ds',
 'Se2C2C2ss',
 'SdO(keto)',
 'Se1C2C2dd',
 'Se2C2O1s',
 'SaaN',
 'SaasN',
 'Se1C1C3a',
 'Se1C2N3sa',
 'SeaC3N2aa',
 'SeaC3N3aa',
 'SeaN2N3aa',
 'Se1C1C2d',
 'Se1C2C3ds',
 'Se1C3C3ss',
 'Se2C2C3ss',
 'SdO(acid)',
 'Se1C2S2ss',
 'Se1C3N2as',
 'Se1C3O1d',
 'Se1C3S2as',
 'SsOH(acid)',
 'SssNH',
 'SssNH(ar)',
 'SssS',
 'Se1C3C3as',
 'Se1C3Cl1s',


# assign indexes based on smiles

In [418]:
mask = df.SMILES.isin(test.SMILES)
df.loc[mask, 'dataset'] = 'test'

mask = df.SMILES.isin(train.SMILES)
df.loc[mask, 'dataset'] = 'train'

In [419]:
df.dataset.value_counts()

dataset
train    1011
test      500
Name: count, dtype: int64

In [420]:
# test.loc[498, 'SMILES']

In [421]:
# df.loc[1509, 'SMILES']

# Reindex based on smiles

In [422]:
df.set_index('SMILES', inplace=True)
train.set_index('SMILES', inplace=True)
test.set_index('SMILES', inplace=True)

In [423]:
idx = pd.Index(np.concatenate([train.index, test.index]), name='SMILES')
idx.shape

(1511,)

In [424]:
df = df.reindex(idx)

In [425]:
df['target'] = train.activity

In [426]:
df.reset_index(inplace=True)

In [427]:
df.head()

,SMILES,ALogPS_logP,ALogPS_logS,HALOG,MW,N.1,NA,NH,O,P,...,SdddN,SdddN(nitro),Se2N3O1d,SddSi,Se2O1Si2d,Se1B3C3sa,Se1N3pO1a,SeaC2N3paa,dataset,target
0,CC1(C)[C@@H]2C[C@H]1C(=C)CC2,3.94,-3.33,0,136.3,0,10,16,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,train,12.3
1,COC(=O)C(C)OC1=CC=C(OC2=CC=C(Cl)C=C2Cl)C=C1,5.13,-5.40,2,341.2,0,22,14,4,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,train,94.1
2,CCCCCCCCCCCCCC(=O)OCC(O)CO,4.77,-4.09,0,302.5,0,21,34,4,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,train,38.1
3,CC(=O)O[C@@H]1C[C@@H]2CC[C@@]1(C)C2(C)C,3.50,-3.52,0,196.3,0,14,20,2,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,train,11.8
4,CCCCCCCCCC=CCC1CC(=O)OC1=O,5.68,-5.73,0,266.4,0,19,26,3,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,train,98.4


# Basic preprocessing

In [428]:
from src.preprocess import preprocess
from src.utils import smi2mol, mol2smi, get_fps_offset


def prepare_data(df, smi_col='SMILES'):
    df['rdmol'] = df[smi_col].apply(smi2mol)
    df.rdmol = df.rdmol.apply(preprocess)
    df['smi'] = df.rdmol.apply(mol2smi)
    return df

In [429]:
df = mem.cache(prepare_data)(df)

# generate representation: fingerprints, rdkit, mordred descriptors

In [437]:
from src.representation import get_representation_from_series

r = mem.cache(get_representation_from_series, ignore=['n_jobs'])(df.smi, n_jobs=N_JOBS)
r.shape

(1511, 4389)

In [438]:
FPS_OFFSET = get_fps_offset(r.columns)
fps_cols = list(range(FPS_OFFSET))

In [439]:
r.dropna(axis=1, inplace=True)
r.shape

(1511, 3280)

In [440]:
mask = (r.iloc[:, FPS_OFFSET:] == 0).all()
zero_cols = r.iloc[:, FPS_OFFSET:].columns[mask]
r.drop(columns=zero_cols, inplace=True)
r.shape

(1511, 3152)

In [441]:
df = pd.concat([df, r], axis=1)
df.shape

(1511, 3523)

In [446]:
rd_cols = [c for c in df.columns if 'rd_' in str(c)]
md_cols = [c for c in df.columns if 'md_' in str(c)]

# Split train test

In [457]:
train = df.query("dataset == 'train'")[fps_cols + rd_cols + md_cols + features]
y_train = df.query("dataset == 'train'").target.values

test = df.query("dataset == 'test'")[fps_cols + rd_cols + md_cols + features]

train.shape, test.shape

((1011, 3518), (500, 3518))

# Model training

In [461]:
import time

import numpy as np
import pandas as pd

from sklearn.linear_model import Lasso, Ridge, LinearRegression
from sklearn.cross_decomposition import PLSRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score, KFold
from src.utils import OffsetScaler, get_fps_offset
import xgboost as xgb
from catboost import CatBoostRegressor
import seaborn as sns


mae = 'neg_mean_absolute_error'
mse = 'neg_mean_squared_error'
rmse = 'neg_root_mean_squared_error'


# prepare models
models = {}

# models['LR'] = LinearRegression()
models['Ridge'] = Ridge()
models['PLS'] = PLSRegression()
models['DT'] = DecisionTreeRegressor(random_state=RANDOM_SEED)
models['Lasso'] = Lasso()
models['KNN'] = KNeighborsRegressor(n_jobs=N_JOBS)
models['SVR'] = SVR()
models['RF'] = RandomForestRegressor(random_state=RANDOM_SEED, n_jobs=N_JOBS)
models['XGB'] = xgb.XGBRegressor(random_state=RANDOM_SEED, n_jobs=N_JOBS, verbosity=0)
models['CATB'] = CatBoostRegressor(loss_function='RMSE', verbose=False, random_seed=RANDOM_SEED, thread_count=N_JOBS)
models['CATB GPU'] = CatBoostRegressor(loss_function='RMSE', verbose=False, random_seed=RANDOM_SEED, thread_count=N_JOBS, task_type='GPU')

# Features + FPS

In [ ]:
X_train = train[fps_cols + rd_cols + md_cols + features]

scaler = OffsetScaler(offset=FPS_OFFSET)

X_train_scale = scaler.fit_transform(X_train.values)

results = {}

for name in models.keys():
    tic = time.time()
    
    model = models[name]
    kfold = KFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)
    results[name] = cross_val_score(model, X_train_scale, y_train, cv=kfold, scoring=rmse)
    
    toc = time.time()    
    print("%5s: %3.3f ± %3.3f    %.1fs" % (name, results[name].mean(), results[name].std(), toc - tic))
    
results = pd.DataFrame(results)
sns.boxplot(results);

Ridge: -49.862 ± 10.061    1.1s
  PLS: -29.369 ± 0.844    0.4s
   DT: -33.885 ± 3.175    1.9s
Lasso: -27.472 ± 0.908    1.9s
  KNN: -27.694 ± 2.103    0.2s
  SVR: -34.691 ± 0.974    5.0s
   RF: -24.100 ± 1.753    19.0s
  XGB: -24.798 ± 1.675    14.9s


# Features only

In [ ]:
X_train = train[features]
X_train_scale = scaler.fit_transform(X_train.values)

results = {}

for name in models.keys():
    tic = time.time()
    
    model = models[name]
    kfold = KFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)
    results[name] = cross_val_score(model, X_train_scale, y_train, cv=kfold, scoring=rmse)
    
    toc = time.time()
    print("%5s: %3.3f ± %3.3f    %.1fs" % (name, results[name].mean(), results[name].std(), toc - tic))
    
results = pd.DataFrame(results)
sns.boxplot(results);

# RandomForest finetune

In [241]:
models['RF 100'] = RandomForestRegressor(n_estimators=100, random_state=RANDOM_SEED, n_jobs=N_JOBS)
models['RF 200'] = RandomForestRegressor(n_estimators=200, random_state=RANDOM_SEED, n_jobs=N_JOBS)
models['RF 400'] = RandomForestRegressor(n_estimators=400, random_state=RANDOM_SEED, n_jobs=N_JOBS)
models['RF 800'] = RandomForestRegressor(n_estimators=400, random_state=RANDOM_SEED, n_jobs=N_JOBS)

In [242]:
X_train = df.query('dataset == "train"')[fps_cols + features]
X_train_scale = scaler.fit_transform(X_train.values)

results = {}

for name in ['RF 100', 'RF 200', 'RF 400', 'RF 800', 'CATB']:
    tic = time.time()
    
    model = models[name]
    kfold = KFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)
    results[name] = cross_val_score(model, X_train_scale, y_train, cv=kfold, scoring=rmse)
    
    toc = time.time()    
    print("%5s: %3.3f ± %3.3f    %.1fs" % (name, results[name].mean(), results[name].std(), toc - tic))
    
results = pd.DataFrame(results)
sns.boxplot(results);

KeyboardInterrupt: 